In [2]:
from PyPDF2 import PdfReader
from langchain.chains import RetrievalQA
# from langchain.vectorstores import Chroma
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain_openai import AzureChatOpenAI
from langchain.chat_models import ChatOpenAI
import openai
import os
import streamlit as st

class Document:
    def __init__(self, file, all_text = "") -> None:
        self.file_name = file.replace(".pdf", "").replace(".", "_")
        self.all_text = all_text
        self.file = file
        openai.api_key = st.secrets['AZURE_OPENAI_API_KEY']
        openai.api_base = st.secrets["AZURE_OPENAI_ENDPOINT"]
        openai.api_type = "azure"
        openai.api_version = "2023-05-15"
        os.environ["AZURE_OPENAI_API_KEY"] = openai.api_key
        os.environ["AZURE_OPENAI_ENDPOINT"] = openai.api_base
        os.environ["OPENAI_API_TYPE"] = openai.api_type
        os.environ["OPENAI_API_KEY"] = openai.api_key
        os.environ["OPENAI_ENDPOINT"] = openai.api_base
 

    def create_chunks(self, chunk_size=1000):
        if self.all_text == "":
            #Read the file
            pdf_reader = PdfReader(self.file)
            all_text = ""
            for page in range(len(pdf_reader.pages)):
                all_text += pdf_reader.pages[page].extract_text() + '\n\n'
            self.all_text = all_text
            print("Document read")
        
        # Split into document objects   
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=20,
            length_function=len,
            is_separator_regex=False,
        )
        chunks = text_splitter.create_documents([self.all_text])
        return chunks 
    
    def get_db(self):
        embedding_model = AzureOpenAIEmbeddings(azure_deployment="text-embedding-ada-002")
        chunks = Document.create_chunks(self)
        vectordb = FAISS.from_documents(chunks, embedding_model)
        return vectordb

In [5]:
document_reader = Document("wounds-and-lacerations-emergency-care-and-closure.pdf")
db = document_reader.get_db()
# db = db.save_local("faiss_index")

# new_db = FAISS.load_local("faiss_index", embeddings)

Document read


In [6]:
db = db.save_local("faiss_index/wounds-and-lacerations-emergency-care-and-closure")

In [9]:
embedding_model = AzureOpenAIEmbeddings(azure_deployment="text-embedding-ada-002")
new_db = FAISS.load_local("faiss_index/wounds-and-lacerations-emergency-care-and-closure", embedding_model, allow_dangerous_deserialization=True)

In [15]:
class Question_Engine: 
    def __init__(self, vectordb) -> None:
        self.vectordb = vectordb
        self.llm = AzureChatOpenAI(deployment_name='gpt-4', 
                      model_name='gpt-4', 
                      temperature=0, 
                      api_version="2023-08-01-preview")
        self.qa_chain = RetrievalQA.from_chain_type(
            llm = self.llm, 
            chain_type="stuff", 
            retriever=self.vectordb.as_retriever(),
            return_source_documents=True,
        )

    def get_answer(self, query):
        llm_response = self.qa_chain({"query": query})
        return llm_response

In [16]:
qEngine =  Question_Engine(new_db)

In [17]:
llm_response = qEngine.get_answer("Que es la anestesia")

c:\Users\jesus\anaconda3\envs\chatbot\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


In [18]:
llm_response

{'query': 'Que es la anestesia',
 'result': 'La anestesia es un método utilizado para prevenir el dolor durante los procedimientos médicos. Funciona al interferir con la transmisión de impulsos nerviosos, lo que evita que el paciente sienta dolor. Los anestésicos locales, como la lidocaína, la mepivacaína y la bupivacaína, se infiltran en los tejidos y difunden a través de las membranas y vainas neurales. Al prevenir la entrada de sodio a través de las membranas nerviosas, se bloquea la conducción del impulso nervioso. Sin impulsos nerviosos, se logra la anestesia. La elección del anestésico y la técnica a utilizar debe ser individualizada para cada paciente, teniendo en cuenta factores como el tipo, la ubicación y la extensión de la herida, así como la duración estimada del procedimiento.',
 'source_documents': [Document(page_content='Effective anesthesia is essential for successful patient intervention and wound repair.\nAs with any procedure, success depends on a thorough understand